# Импорты


In [ ]:
!gdown 1tZq9BPXcG-YQkV40ryIwVqPQUpWXCk1- -O /content/test/
!gdown 12hNXcrHr0v48m9VRr-GLj5eq3pnYU7Im
!unzip -qq /content/test/test_dataset_mc2.zip -d /content/test

Downloading...
From: https://drive.google.com/uc?id=1tZq9BPXcG-YQkV40ryIwVqPQUpWXCk1-
To: /content/test/test_dataset_mc2.zip
100% 876M/876M [00:05<00:00, 151MB/s] 
Downloading...
From: https://drive.google.com/uc?id=12hNXcrHr0v48m9VRr-GLj5eq3pnYU7Im
To: /content/requirements.txt
100% 225/225 [00:00<00:00, 443kB/s]
replace /content/test/__MACOSX/._eye_test? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install -r /content/requirements.txt

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

import pandas as pd 
import numpy as np
import glob
import multiprocessing
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
import segmentation_models_pytorch as smp

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.models import resnet18
from torchvision.utils import draw_segmentation_masks

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")
import json
from GPUtil import showUtilization as gpu_usage
import GPUtil

# Датасет

Прежде чем разбираться с моделями, нам надо в первую очередь разобраться с тем, как грузить датасет. Давайте напишем класс в торче для этого.

## Класс датасета

In [ ]:
class EyeDataset(Dataset):
    """
    Класс датасета, организующий загрузку и получение изображений и соответствующих разметок
    """

    def __init__(self, data_folder: str, mode: str = "train", transform=None):
        self.class_ids = {"vessel": 1}

        self.mode = mode.lower()

        self.data_folder = data_folder
        self.transform = transform
        self._image_files = glob.glob(f"{data_folder}/*.png")
        if self.mode == "train":
            self._mask_files = glob.glob(f"{data_folder}/*.geojson")

        image_file_ids = set([el.split(".")[0] for el in self._image_files])
        if self.mode == "train":
            mask_file_ids = set([el.split(".")[0] for el in self._mask_files])

        if self.mode == "train":
            intersecting_ids = list(image_file_ids.intersection(mask_file_ids))
            self._image_files = [el + ".png" for el in intersecting_ids]
            self._mask_files = [el + ".geojson" for el in intersecting_ids]

    @staticmethod
    def read_image(path: str) -> np.ndarray:
        image = cv2.imread(str(path), cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.array(image / 255, dtype=np.float32)
        return image

    @staticmethod 
    def parse_polygon(coordinates, image_size): 
        mask = np.zeros(image_size, dtype=np.float32) 
    
        if len(coordinates) == 1: 
            points = [np.int32(coordinates)] 
            cv2.fillPoly(mask, points, 1) 
        else: 
            points = [np.int32([coordinates[0]])] 
            cv2.fillPoly(mask, points, 1) 
    
            for polygon in coordinates[1:]: 
                points = [np.int32([polygon])] 
                cv2.fillPoly(mask, points, 0) 
    
        return mask

    @staticmethod
    def parse_mask(shape: dict, image_size: tuple) -> np.ndarray:
        """
        Метод для парсинга фигур из geojson файла
        """
        mask = np.zeros(image_size, dtype=np.float32)
        coordinates = shape['coordinates']
        if shape['type'] == 'MultiPolygon':
            for polygon in coordinates:
                mask += EyeDataset.parse_polygon(polygon, image_size)
        else:
            mask += EyeDataset.parse_polygon(coordinates, image_size)

        return mask

    def read_layout(self, path: str, image_size: tuple) -> np.ndarray:
        """
        Метод для чтения geojson разметки и перевода в numpy маску
        """
        with open(path, 'r', encoding='cp1251') as f:  # some files contain cyrillic letters, thus cp1251
            json_contents = json.load(f)

        num_channels = 1 + max(self.class_ids.values())
        mask_channels = [np.zeros(image_size, dtype=np.float32) for _ in range(num_channels)]
        mask = np.zeros(image_size, dtype=np.float32)

        if type(json_contents) == dict and json_contents['type'] == 'FeatureCollection':
            features = json_contents['features']
        elif type(json_contents) == list:
            features = json_contents
        else:
            features = [json_contents]

        for shape in features:
            channel_id = self.class_ids["vessel"]
            mask = self.parse_mask(shape['geometry'], image_size)
            mask_channels[channel_id] = np.maximum(mask_channels[channel_id], mask)

        mask_channels[0] = 1 - np.max(mask_channels[1:], axis=0)

        return np.stack(mask_channels, axis=-1)


    def __getitem__(self, idx: int) -> dict:
        # Достаём имя файла по индексу
        image_path = self._image_files[idx]

        # Получаем соответствующий файл разметки
        if self.mode == "train":
            json_path = image_path.replace("png", "geojson")

        image = self.read_image(image_path)

        if self.mode == "train":
            mask = self.read_layout(json_path, image.shape[:2])

        if self.mode == "train":
            sample = {'image': image,
                      'mask': mask}
        else:
            sample = {"image": image}

        if self.transform is not None:
            sample = self.transform(**sample)

        return sample

    def __len__(self):
        return len(self._image_files)

    # Метод для проверки состояния датасета
    def make_report(self):
      reports = []
      if (not self.data_folder):
        reports.append("Путь к датасету не указан")
      if (len(self._image_files) == 0):
        reports.append("Изображения для распознавания не найдены")
      else:
        reports.append(f"Найдено {len(self._image_files)} изображений")
      cnt_images_without_masks = sum([1 - len(glob.glob(filepath.replace("png", "geojson"))) for filepath in self._image_files])
      if cnt_images_without_masks > 0:
        reports.append(f"Найдено {cnt_images_without_masks} изображений без разметки")
      else:
        reports.append(f"Для всех изображений есть файл разметки")
      return reports


class DatasetPart(Dataset):
    """
    Обертка над классом датасета для его разбиения на части
    """
    def __init__(self, dataset: Dataset,
                 indices: np.ndarray,
                 transform: A.Compose = None):
        self.dataset = dataset
        self.indices = indices

        self.transform = transform

    def __getitem__(self, idx: int) -> dict:
        sample = self.dataset[self.indices[idx]]

        if self.transform is not None:
            sample = self.transform(**sample)

        return sample

    def __len__(self) -> int:
        return len(self.indices)


# Предикшн

## Класс Предиктора

In [ ]:
import gc
class UnetPredictor:
    def __init__(self, model: nn.Module, device: str):
        self.model = model
        self.device = torch.device(device)
        self.model.to(self.device)

    def predict(self, dataloader: torch.utils.data.DataLoader) -> np.ndarray:
        self.model.eval()

        preds = []
        print(len(dataloader))
        with torch.no_grad():
            for batch in tqdm(dataloader):
                image = batch["image"].to(self.device)
                output = self.model(image).cpu()
                preds.append(torch.exp(output[:,1]))
                del output
                gc.collect()
        return torch.cat(preds, dim=0).numpy()

##Предсказание

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
test_data = '/content/test/eye_test'

In [8]:
model = torch.load('/content/drive/MyDrive/Competitions/LeadersOfDigital/models/pretrain/model.pt')
#model_2 = torch.load('/content/drive/MyDrive/Competitions/LeadersOfDigital/models/model_1024px_10x_1438.pt')

In [9]:
cores=multiprocessing.cpu_count()
size= 1024
eval_aug = A.Compose([
    A.LongestMaxSize(size, interpolation=cv2.INTER_CUBIC),
    A.PadIfNeeded(size, size),
    ToTensorV2(transpose_mask=True)
])

transforms = {"test": eval_aug}

test_dataset = EyeDataset(
    test_data, mode='predict', transform=transforms['test']
)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=1,
                                            num_workers=cores-1,
                                            shuffle=False)

predictor = UnetPredictor(model=model, device="cuda")
#predictor2 = UnetPredictor(model=model_2, device="cuda")
preds = predictor.predict(test_loader)
del test_loader
del eval_aug
gc.collect()
#preds_2 = predictor2.predict(test_loader)

301


100%|██████████| 301/301 [02:39<00:00,  1.89it/s]


0

In [10]:
eval_aug = A.Compose([
    A.LongestMaxSize(size, interpolation=cv2.INTER_CUBIC),
    A.PadIfNeeded(size, size),
    A.HorizontalFlip(always_apply=True),
    ToTensorV2(transpose_mask=True)
])

transforms = {"test": eval_aug}
test_dataset = EyeDataset(
    test_data, mode='predict', transform=transforms['test']
)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=1,
                                            num_workers=cores-1,
                                            shuffle=False)
predictor = UnetPredictor(model=model, device="cuda")
#predictor2 = UnetPredictor(model=model_2, device="cuda")
preds_fliph = predictor.predict(test_loader)
del test_loader
del eval_aug
gc.collect()
#preds_2 = predictor2.predict(test_loader)

301


100%|██████████| 301/301 [02:35<00:00,  1.93it/s]


0

In [11]:
eval_aug = A.Compose([
    A.LongestMaxSize(size, interpolation=cv2.INTER_CUBIC),
    A.PadIfNeeded(size, size),
    A.VerticalFlip(always_apply=True),
    ToTensorV2(transpose_mask=True)
])

transforms = {"test": eval_aug}
test_dataset = EyeDataset(
    test_data, mode='predict', transform=transforms['test']
)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=1,
                                            num_workers=cores-1,
                                            shuffle=False)
predictor = UnetPredictor(model=model, device="cuda")
#predictor2 = UnetPredictor(model=model_2, device="cuda")
preds_flipv = predictor.predict(test_loader)
del test_loader
del eval_aug
#preds_2 = predictor2.predict(test_loader)
gc.collect()

301


100%|██████████| 301/301 [02:36<00:00,  1.92it/s]


0

In [12]:
eval_aug = A.Compose([
    A.LongestMaxSize(size, interpolation=cv2.INTER_CUBIC),
    A.PadIfNeeded(size, size),
    A.VerticalFlip(always_apply=True),
    A.HorizontalFlip(always_apply=True),
    ToTensorV2(transpose_mask=True)
])
transforms = {"test": eval_aug}
test_dataset = EyeDataset(
    test_data, mode='predict', transform=transforms['test']
)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=1,
                                            num_workers=cores-1,
                                            shuffle=False)
predictor = UnetPredictor(model=model, device="cuda")
#predictor2 = UnetPredictor(model=model_2, device="cuda")
preds_flipvh = predictor.predict(test_loader)
del test_loader
del eval_aug
gc.collect()
#preds_2 = predictor2.predict(test_loader)

301


100%|██████████| 301/301 [02:37<00:00,  1.91it/s]


0

In [13]:
transform = A.Compose([
    A.HorizontalFlip(p=1),
])
preds_fliph = transform(image=preds_fliph)['image']
del transform
gc.collect()

22

In [14]:
transform = A.Compose([
    A.VerticalFlip(p=1)
])
preds_flipv = transform(image=preds_flipv)['image']
del transform
gc.collect()

22

In [15]:
transform = A.Compose([
    A.VerticalFlip(p=1),
    A.HorizontalFlip(p=1)
])
preds_flipvh = transform(image=preds_flipvh)['image']
del transform
gc.collect()

44

In [16]:
round_preds=(preds+preds_fliph+preds_flipv+preds_flipvh)/4

In [17]:
'''transform = A.Compose([
    A.VerticalFlip(p=1),
])
round_preds = transform(image=round_preds)['image']
del transform
gc.collect()'''

44

In [18]:
treshhold = 0.5
round_preds[round_preds>=treshhold]=255
round_preds[round_preds<treshhold]=0

In [19]:
!mkdir /content/preds

mkdir: cannot create directory ‘/content/preds’: File exists


In [20]:
from PIL import Image
quality = 0.1
box = (0, (size-777)/2, size, size-(size-777)/2)
for i in range(len(round_preds)):
    im = Image.fromarray(round_preds[i])
    im = im.convert('1')
    im = im.crop(box)
    im = im.resize((1624,1232))
    im.save(test_dataset._image_files[i].replace("/content/test/eye_test", '/content/preds'),compress_level=9 ,bits=2, format="PNG")

In [21]:
!zip -9 -q -r test_submit.zip /content/preds/